In [1]:
import cobra
import GEMS
import pandas as pd
import numpy as np
import os
import re

import pickle

from etcpy import etc
from etcpy import tempDep

from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt

In [2]:
data = GEMS.load_exp_data('../data/ExpGrowth.csv')
path = '../'
params = pd.read_csv(os.path.join(path,'data/model_enzyme_params_new_tagged.csv'),index_col=0)
df = etc.calculate_thermal_params(params)

dfae_batch = data[0].set_index('Ts').rename_axis(None)
params

,Topt,Topt_std,Length,Tm,Tm_std,T90,dCpt,dCpt_std,topt_source,TmTag
P77467,306.15,10,262,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76077,311.15,10,309,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76078,310.15,10,95,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
P76079,309.15,10,248,328.717325,7.588735,NaN,-4000,1000,predicted,Mean
P76081,310.15,10,356,328.717325,7.588735,NaN,-4000,1000,BullShit,Mean
...,...,...,...,...,...,...,...,...,...,...
P0AG80,313.15,10,438,329.166303,4.070000,330.45,-4000,1000,predicted,Exp
P77718,307.15,10,482,319.125751,4.070000,320.15,-4000,1000,predicted,Exp
P0A6D5,308.15,10,288,328.762604,7.710000,NaN,-4000,1000,predicted,Mean
standard,308.15,10,272,328.762604,7.710000,NaN,-4000,1000,BullShit,Mean


In [3]:
def aerobic(params):
    # thermalParams: a dictionary with ids like uniprotid_Topt 
    df = etc.calculate_thermal_params(params)
    #mae = pickle.load(open(os.path.join(path,'models/aerobic.pkl'),'rb'))
    mae = cobra.io.load_matlab_model('../models/ecModel.mat')
    
    match = re.compile('usage_prot_*', re.IGNORECASE)
    for react in mae.reactions:
        if match.match(react.id) and react.lower_bound != -1000:
            ori_lb = react.lower_bound
            react.lower_bound = ori_lb*2.5


    mae.reactions.EX_glc__D_e.lower_bound = -100
    mae.reactions.BIOMASS_Ec_iJO1366_core_53p95M.upper_bound = 50
    #mae.medium['EX_glc__D_e'] = 100
    
    try: 
        rae, gae, oae, cae, aae = etc.simulate_growth(mae,dfae_batch.index+273.15,df=df,sigma=0.5)
    except: 
        rae = np.zeros(dfae_batch.shape[0])
        model = None
    
    rae = [0 if x is None else x for x in rae]
    rae = [0 if x<1e-3 else x for x in rae]
    rexp = dfae_batch['r'].values
    #print(rae)
    print(abs(rae - dfae_batch['r'].values))
    print('r2_batch:',r2_score(rexp,rae))
    print('MSE_ae',MSE(rexp,rae))
    return {'growth':np.array(rae), 'glucose':np.array(gae), 'oxygen': np.array(oae), 'carbonDioxide': np.array(cae), 'acetate': np.array(aae)}, r2_score(rexp,rae), MSE(rexp,rae)

In [4]:
S = 0
SMin = 0
df = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])

for i in range(1,11):
    print(i)
    if i == 1:
        params = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Topt', 'dCpt'])
        rae, score, mse = aerobic(params)

    else:
        params = tempDep.sample_data_uncertainty_with_constraint_small_increasing_topt(params, ['Topt', 'dCpt'])
        rae, score, mse = aerobic(params)
    
    if score > S:
        S = score
        params.to_csv('../data/BestParamsTopt.csv')
        bestParams = params

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-06
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.40343806687743
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.5941458504516964
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.8577399221169909
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  1.0162000136367155
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  1.1551641384266285
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.2761886185039346
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.4053436902361276
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.5109118450348393
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.413208889285365
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3109573593200925
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.173236435912858
NGAM is: 6.9000236503978725
Growth at  44.4534910975740

In [5]:
S = 0
SMin = 0
params = bestParams

for i in range(1,51):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score, mse = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv("../data/BestParams" + str(i) + ".csv")
        bestParams = newParams
    
    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParams" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsTm1.csv')
else:
    bestParams.to_csv('../data/BestParamsTm1.csv')

1
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.2083372721379392
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.3493968783720025
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.5808348570492758
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.7439105171778456
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.9055302364905723
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.0628352920629134
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.2548107064177887
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4204262784792705
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4853061792115807
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.431051890867307
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3041170698883289
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.151049526818284
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.046224811153879525
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.11216692999802542
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.1805123992908704
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.2720416746277202
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.39187608281990427
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.5987258518403962
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.0671204099989744
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4731992567615824
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4766802364836271
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.349980886702998
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2900378810510431
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


...

Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  28.251819864188008 is:  0
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.3818163804510094
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.7665154829961484
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.979205505601049
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3064136315209762
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4103000283549383
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4612814984868105
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.309587396928063
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1125680569326541
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.0385650583374364
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:  0.9212782676616114
NGAM is: 7.226932905456837
Growth at  48.47717620988101 is:  0.7710824153444248
[0.27036    0.58194    0.636804   0.78318    0.57427162 

In [6]:
S = 0
SMin = 0
params = bestParams

for i in range(1,51):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt', 'dCpt'])
    rae, score, mse = aerobic(newParams)
    
    if score > S:
        S = score
        params.to_csv("../data/BestParamsTopt" + str(i) + ".csv")
        bestParams = newParams

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsToptFinal1.csv')
else:
    bestParams.to_csv('../data/BestParamsToptFinal1.csv')

1
NGAM is: 3.23


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.25348400458617776
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.49294476654142677
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6476987612228676
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.801594682696807
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9490696937538283
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.126094678234154
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.36580179577831
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4748554465203005
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.513856735717676
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3893149815167278
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2084730319377597
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.13927

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.2638737806629886
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.4548316288270039
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.5999039559160263
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.7537496900901696
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9150263859647526
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1317168669079374
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4078398405194172
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4711959541476218
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4547690882856557
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3417215961131235
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2029546137515459
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.14

...

Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.1556322709673586
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.2926976192798083
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.4062726029265742
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5345920323045078
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6773124435154714
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.881994189680513
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.2356091807543799
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4583495865012126
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4795184972572124
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3189861710853852
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1374829709557188
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.069

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.4547727265314003
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6047525172821103
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.7587574367675154
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.913160896224441
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1071221524356372
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3746063735159162
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4790206756558066
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4916984444338812
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3344184356707611
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1558236938961937
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.0882406795076702
NGAM is: 7.088333029723301
Growth at  46.6453309881680

In [7]:
S = 0
SMin = 0
params = bestParams

for i in range(51, 101):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score, mse = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv("../data/BestParams" + str(i) + ".csv")
        bestParams = newParams
    
    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParams" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsTm2csv')
else:
    bestParams.to_csv('../data/BestParamsTm2csv')

51
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.17132006872457
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.2962897829867001
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.5079111941647195
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6604901932857037
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8139273838996794
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9557193240222492
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.121923671443005
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3553878750230517
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4593185046637187
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4728127657961034
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3393755775895597
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1572731844575306
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.047968337350282726
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.2136519681108976
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.30023652247313365
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.40047949257244914
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.5151437998648051
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.6858401597336847
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.0003882780353204
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.255367947282198
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4743506511985947
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.374343416019212
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2488199283829358
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


...

Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  28.251819864188008 is:  0
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5132129105555328
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.7004188001889051
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.9219813969264664
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.2751334930406377
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.415713017840866
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4768167342582215
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.4327401441648941
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.3220169596838678
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.263991764720433
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:  1.1692117063433929
NGAM is: 7.226932905456837
Growth at  48.47717620988101 is:  1.0417747996559
[0.27036    0.58194    0.636804   0.78318    0.44287509 0.3

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.25086105714673956
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.598029639004374
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.7676410148488414
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.894215233661687
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.0056675090980884
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1477467703941078
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3486702613087704
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4474350475985538
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.5096974588567507
NGAM is: 6.681009354243395
Failed to solve the problem
Growth at  42.23876395989504 is:  0
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.473591850715955
NGAM is: 6.974987953458825
Growth at  45.28989924561603 

In [8]:
S = 0
SMin = 0
params = bestParams

for i in range(51,101):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt', 'dCpt'])
    rae, score, mse = aerobic(newParams)
    
    if score > S:
        S = score
        params.to_csv("../data/BestParamsTopt" + str(i) + ".csv")
        bestParams = newParams

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsToptFinal2.csv')
else:
    bestParams.to_csv('../data/BestParamsToptFinal2.csv')

51
NGAM is: 3.23


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.022870406221994363
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.05874907581690282
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.09904239262434666
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.15699683209675422
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.2387542359044896
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.39337673886697544
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.7973459080799831
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.2630576973089556
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.368105244763409
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.2271031505120482
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.0623497616053732
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is: 

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


...

Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.11794904219039223
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.26108228422547414
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.3973217254285908
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5681461017553806
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.778052816266992
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1151449824351847
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.35282982505067
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4469897648137648
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4303051885531324
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3107214788482247
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1415188542286119
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.075

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.41437835335484174
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6607743445197083
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8365796668366469
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9920388900848437
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1613900271448092
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3716939786415883
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4598660589957868
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4984254020328014
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.41040510833933
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2972014345466207
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.2508552683540193
NGAM is: 7.088333029723301
Growth at  46.6453309881680

In [9]:
S = 0
SMin = 0
params = bestParams

for i in range(101, 151):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score, mse = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv("../data/BestParams" + str(i) + ".csv")
        bestParams = newParams
    
    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParams" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsTm3.csv')
else:
    bestParams.to_csv('../data/BestParamsTm3.csv')

101
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.004465607298874174
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.012216939662213048
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.03700693247297926
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.06786692979491454
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.11534657584190416
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.1861727953665742
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.3275819389111269
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.7211919068143059
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.1970865142013178
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4910368383466248
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3460900991522124
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1577353525228813
NGAM is: 6.974987953458825
Growth at  45.2898992456

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  20.153299075508016 is:  0
NGAM is: 3.23


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.450404056353102
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.7297934579666949
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8907342969987554
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.041988764890906
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.215148731890926
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4248405645515
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.5028006912500433
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4716113738652303
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3673786019725482
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2357475432603493
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.1826775558161307
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:

...

Failed to solve the problem
Growth at  23.00151111308503 is:  0
NGAM is: 3.666948749964016


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  28.251819864188008 is:  0
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.42598354288499085
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.8421910363615162
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1434163369542036
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3644742090189106
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4557705121715765
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4128755953271646
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.2779262935783513
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1175636068127943
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.0556820046944364
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:  0.4987948185853894
NGAM is: 7.226932905456837
Growth at  48.47717620988101 is:  0.16586581092144917
[0.27036    0.58194    0.636804   0.78318    0.53010

In [10]:
S = 0
SMin = 0
params = bestParams

for i in range(101,151):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt', 'dCpt'])
    rae, score, mse = aerobic(newParams)
    
    if score > S:
        S = score
        params.to_csv("../data/BestParamsTopt" + str(i) + ".csv")
        bestParams = newParams

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsToptFinal3.csv')
else:
    bestParams.to_csv('../data/BestParamsToptFinal3.csv')

101
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.0
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.4108730445876131
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.6345278777477239
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.7779862487102689
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.915373076254708
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.0445920559655597
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1965871590313342
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3915154461258965
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4604805173823057
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4487885659296682
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3415766852587967
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.2023854422769698
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.13342641947

In [11]:
S = 0
SMin = 0
params = bestParams

for i in range(151, 201):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score, mse = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv("../data/BestParams" + str(i) + ".csv")
        bestParams = newParams
    
    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParams" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsTm4.csv')
else:
    bestParams.to_csv('../data/BestParamsTm4.csv')

151
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.2152320206939853
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.3727583000606538
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.6457570489071114
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.8484054881132058
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.9948900066623221
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  1.115576083010019
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.2563925239521827
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4355415616065774
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4440945770322826
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3334852150983607
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.1830302594501414
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.0132366269929152
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  48.47717620988101 is:  0
[0.27036    0.5801932  0.62897992 0.76529537 0.91908314 1.02076424
 1.02891784 0.84362334 0.47277736 0.10805409 0.07870568 0.2099345
 0.10902019 0.28223438 0.16272   ]
r2_batch: -1.3614843016903824
MSE_ae 0.36343585265840317
154
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.0004539642198992278
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.0020889541923954052
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.011414932283461904
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.029163201252410387
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.06661276609309648
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.14108246580291894
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.3443497455984603
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.2132711278569017
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4057256447351363

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  48.47717620988101 is:  0
[0.27036    0.57985105 0.62538907 0.7540168  0.88947523 0.95126553
 0.84127425 0.10288887 0.03330236 0.15123633 0.01738184 0.79003237
 0.67548508 0.74809795 0.16272   ]
r2_batch: -1.3713091111908953
MSE_ae 0.36494790506352365
155
NGAM is: 3.23
Growth at  20.153299075508016 is:  8.46569228804718e-05
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.00042508411345567964
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.002575368632049792
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.006986677211223392
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.016857433661050757
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.03758874079695637
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.09775180470801952
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  0.3783449492000735
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  0.9138861536130

...

Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  28.251819864188008 is:  0
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.6392305558076375
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.8985295673036398
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1300982289430876
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3749897416219363
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4659865034213981
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3859894129647023
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.2546255444660268
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1011474853517151
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1.0423048513592696
NGAM is: 7.088333029723301
Growth at  46.64533098816804 is:  0.37723512771824197
NGAM is: 7.226932905456837


/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  48.47717620988101 is:  0
[0.27036    0.58194    0.636804   0.78318    0.31685744 0.19381843
 0.05552577 0.05882974 0.0269585  0.06741741 0.13501046 0.27981251
 0.03527685 0.54533687 0.16272   ]
r2_batch: 0.13275387644217473
MSE_ae 0.1334704338937663
180
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.007515348801456354
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.04342664022200494
NGAM is: 3.666948749964016
Failed to solve the problem
Growth at  26.331896319367047 is:  0
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.5792011430059553
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8060341510733686
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9908569229065765
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1814018199910117
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.395352789621451
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.482538108949804

In [12]:
S = 0
SMin = 0
params = bestParams

for i in range(151,201):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt', 'dCpt'])
    rae, score, mse = aerobic(newParams)
    
    if score > S:
        S = score
        params.to_csv("../data/BestParamsTopt" + str(i) + ".csv")
        bestParams = newParams

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsToptFinal4.csv')
else:
    bestParams.to_csv('../data/BestParamsToptFinal4.csv')

151
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.17614707955651984
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.31414030017365446
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.5533287447752714
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.6887685066068672
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.8240212411929214
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.9534420334512077
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  1.1149195348370278
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.3378094950328818
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.453652494126
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.495457183053292
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3112728233096391
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.102093234328156
NGAM is: 6.974987953458825
Growth at  45.28989924561603 is:  1

In [13]:
S = 0
SMin = 0
params = bestParams

for i in range(201, 251):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_increasing_topt(params, ['Tm', 'dCpt'])
    rae, score, mse = aerobic(newParams)

    if score > S:
        S = score
        params.to_csv("../data/BestParams" + str(i) + ".csv")
        bestParams = newParams
    
    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParams" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsTm5.csv')
else:
    bestParams.to_csv('../data/BestParamsTm5.csv')

201
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.02563648036122446
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.06140539766190614
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.15637576388768437
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.25727733050820356
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.3941791550787103
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.5749909313531931
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.8884863685884007
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.4162179805441562
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.4460259714574037
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.3409717862250894
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.1966501106161282
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.0315072193462034
NGAM is: 6.974987953458825
Growth at  45.2898992456160

/home/adi/anaconda3/lib/python3.11/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


Failed to solve the problem
Growth at  48.47717620988101 is:  0
[2.55556312e-01 5.43353916e-01 5.26408047e-01 5.87326435e-01
 6.33125780e-01 5.85649981e-01 3.24635588e-01 5.91747276e-04
 1.43902800e+00 1.38107992e-01 2.52071387e-02 2.01801091e-01
 1.01635653e-01 7.32299142e-02 1.62720000e-01]
r2_batch: -0.7349751268582418
MSE_ae 0.2670151836789635
212
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.044564102475129874
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.08783057508700019
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.18083052505189165
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.2651154902699051
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.36748274590388047
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.48995532883077747
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.6815713359247653
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.060359598305999
NGAM is: 6.23740324562199

In [14]:
S = 0
SMin = 0
params = bestParams

for i in range(201, 251):
    print(i)
    newParams = tempDep.sample_data_uncertainty_with_constraint_random_topt(params, ['Topt', 'dCpt'])
    rae, score, mse = aerobic(newParams)
    
    if score > S:
        S = score
        params.to_csv("../data/BestParamsTopt" + str(i) + ".csv")
        bestParams = newParams

    elif score < SMin:
        SMin = score
        params.to_csv("../data/WorstParamsTopt" + str(i) + ".csv")
    
    df1 = pd.DataFrame(columns=['iter', 'temp', 'growth', 'o2flux', 'gluFlux', 'co2flux', 'acetateFlux', 'R2', 'MSE'])
    df1['iter'] = [i for j in range(len(dfae_batch.index))]
    df1['temp'] = dfae_batch.index
    df1['growth'] = rae['growth']
    df1['o2flux'] = rae['oxygen']
    df1['gluFlux'] = rae['glucose']
    df1['co2flux'] = rae['carbonDioxide']
    df1['acetateFlux'] = rae['acetate']
    df1['R2'] = [score for j in range(len(dfae_batch.index))]
    df1['MSE'] = [mse for j in range(len(dfae_batch.index))]
    df = pd.concat([df, df1])

if 'bestParams' not in globals():
    bestParams = newParams
    bestParams.to_csv('../data/BestParamsToptFinal5.csv')
else:
    bestParams.to_csv('../data/BestParamsToptFinal5.csv')

201
NGAM is: 3.23
Growth at  20.153299075508016 is:  0.1225902050162823
NGAM is: 3.23
Growth at  23.00151111308503 is:  0.2030919909429014
NGAM is: 3.666948749964016
Growth at  26.331896319367047 is:  0.3438302791559263
NGAM is: 4.228097946953468
Growth at  28.251819864188008 is:  0.45191017188785415
NGAM is: 4.675583853845237
Growth at  29.99421176442104 is:  0.5685000492725042
NGAM is: 5.047267123847456
Growth at  31.62212188446705 is:  0.6935687803933894
NGAM is: 5.44939156263788
Growth at  33.61728633658504 is:  0.8673839433248103
NGAM is: 5.950125048399887
Growth at  36.55299482796005 is:  1.1591278515488428
NGAM is: 6.237403245621997
Growth at  38.54155004207803 is:  1.3748767784443896
NGAM is: 6.449817187489966
Growth at  40.20094365636703 is:  1.4937057222050962
NGAM is: 6.681009354243395
Growth at  42.23876395989504 is:  1.3825465278033027
NGAM is: 6.9000236503978725
Growth at  44.453491097574044 is:  1.1974886968244067
NGAM is: 6.974987953458825
Growth at  45.28989924561603 i

In [15]:
df.to_csv('../results/modelResultsRun2.csv')